In [6]:
'''
03_create-paths.ipynb
Create a dataset of shortest paths between all destinations
'''

import requests
import pathlib
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import nearest_points, unary_union
import math
import swifter
import json
import geojson
import gdal
import h3
import networkit as nk

PATH_ROOT = os.path.join(pathlib.Path().absolute(), '../..' )
PATH_IN = PATH_ROOT + '/data/02_processed/'
PATH_OUT = PATH_ROOT + '/data/03_paths/'

# constants
CONST_graph_detail_lvl = 8
CONST_speed_day_km = 40
CONST_slope_effect_multiplier = 2
CONST_distance_bridge = 10 / 111
CONST_river_value = 9999

# critical slope algorithm
CONST_critical_slope = 8

# for tobler application
CONST_elevation_coefficient = 1

def slope_coeff(slope):
    return 1 / (1 + ((abs(slope) * 100) / CONST_critical_slope) ** 2)

def calculate_path_time(value_from, value_to, dist):
    rise = (value_to - value_from) / 1000
    slope = rise / dist
    slope_c = slope_coeff(slope)
    time = CONST_speed_day_km * abs(slope_c ** CONST_slope_effect_multiplier)

    el_dist = math.sqrt((rise ** 2) + (dist ** 2))
    path_time = (el_dist / time)

    #print(value_to, value_from, slope, time, el_dist, path_time)
    return path_time

#print(calculate_path_time(100, 300, 2))
#print(calculate_path_time(100, 200, 1))
print(calculate_path_time(50, 200, 40))
print(calculate_path_time(200, 50, 40))

hex_distance = 2 * h3.edge_length(CONST_graph_detail_lvl)
print('hex_distance', hex_distance)

# load datasets

destinations = gpd.read_file(PATH_IN + 'destinations.geojson')

rivers_df = gpd.read_file(PATH_IN + 'rivers.geojson') 
rivers = unary_union(
    [river['geometry'] for ri, river in rivers_df.iterrows()]
)

bridges_df = gpd.read_file(PATH_IN + 'bridges.geojson')
bridges = unary_union(
    [bridge['geometry'].buffer(hex_distance / 222) for bi, bridge in bridges_df.iterrows()]
)

bbox = gpd.read_file(PATH_IN + 'bbox.geojson') 
bb_xy = bbox.total_bounds

bounds_json = dict(
    geojson.Polygon(
        [[
            [bb_xy[0], bb_xy[1]],
            [bb_xy[0], bb_xy[3]],
            [bb_xy[2], bb_xy[3]],
            [bb_xy[2], bb_xy[1]],
            [bb_xy[0], bb_xy[1]],
        ]]
    ))


1.0044064213843935
1.0044064213843935
hex_distance 0.922709368


In [7]:
# load elevation

elevation = gdal.Open(PATH_IN + 'elevation.tif')
band = elevation.GetRasterBand(1)
cols = elevation.RasterXSize
rows = elevation.RasterYSize

transform = elevation.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]

elevation_data = band.ReadAsArray(0, 0, cols, rows)

# check whether there is river or a bridge
def path_value_point (hex_center):
    #return elevation_point(hex_center)
    #on_river = hex.intersects(rivers)
    on_river = hex_center.distance(rivers) < hex_distance / 200
    if on_river: 
        on_bridge = hex_center.distance(bridges) < hex_distance / 200
        if not on_bridge:
            value = CONST_river_value
        else:
            value = elevation_point(hex_center)
    else:
        value = elevation_point(hex_center)
    return value

# get elevation value for the given geographical point
def elevation_point (point):
    row = int((yOrigin - point.y ) / pixelHeight)
    col = int((point.x - xOrigin) / pixelWidth)
    
    if elevation_data.shape[0] > row and elevation_data.shape[1] > col:
        el_value = int(elevation_data[row][col])
        if el_value <= 0: # sea
            return CONST_river_value
        else:
            return el_value
    else:
        return CONST_river_value

In [8]:
# construct hexes dataframe

hex_ids = list(h3.polyfill(bounds_json, CONST_graph_detail_lvl))

hexes_df = pd.DataFrame(hex_ids, columns=['id'])
hexes_df['center'] = hexes_df.swifter.apply(
    lambda x: Point(h3.h3_to_geo(x['id'])),
    axis = 1
)
hexes_df["value"] = hexes_df.swifter.apply(
    lambda x: path_value_point(x['center']),
    axis = 1
)
hexes_df.set_index('id', inplace=True)

# find neighboring hexes
hexes_df['neighbors'] = hexes_df.swifter.apply(
    lambda x: \
        [
            h3.get_destination_h3_index_from_unidirectional_edge(n) 
            for n in h3.get_h3_unidirectional_edges_from_hexagon(x.name)\
        ],
    axis = 1
)

# save hexes_df
hexes_df.to_csv(PATH_OUT + 'hexes.csv')

Pandas Apply: 100%|██████████| 1711237/1711237 [00:48<00:00, 35482.19it/s]


In [9]:
hexes_df

,center,value,neighbors
id,,,
8863a4185bfffff,POINT (14.8745390739082 52.32821401808984),71,"[8863a41ae7fffff, 8863a41859fffff, 8863a41851f..."
885274069dfffff,POINT (20.19757190887843 48.51922149557049),306,"[8852740683fffff, 8852740699fffff, 885274068bf..."
88524698edfffff,POINT (18.42761199331396 49.7557567142236),293,"[88524698e5fffff, 885246983bfffff, 8852469833f..."
885269188bfffff,POINT (18.97688135148771 54.06097202831151),1,"[885269189dfffff, 8852691889fffff, 8852691881f..."
8863b390a5fffff,POINT (12.08361435781251 50.61263651585706),421,"[8863b3919bfffff, 8863b39199fffff, 8863b391d3f..."
...,...,...,...
8863147805fffff,POINT (15.80935339937383 54.54591998078079),9999,"[8863147863fffff, 8863147829fffff, 8863147801f..."
8852750413fffff,POINT (19.96121564276257 49.49717456188855),633,"[885275041bfffff, 88527504e9fffff, 88527504cdf..."
885245d1b5fffff,POINT (18.87294493709587 51.19437328851778),172,"[885245c349fffff, 885245d1bdfffff, 885245d1b1f..."


In [10]:
# create weighted non-directional graph for all combinations of neighboring hexes

from ast import literal_eval

hexes_df = gpd.read_file(
    PATH_OUT + 'hexes.csv'
)
hexes_df['center'] = hexes_df['center'].apply(wkt.loads)
hexes_df['neighbors'] = hexes_df['neighbors'].apply(literal_eval)
hexes_df["value"] = pd.to_numeric(hexes_df["value"])
hexes_df.set_index('id', inplace=True)

g = nk.Graph(directed=False, weighted=True)

hexes_df['node'] = hexes_df.apply(
    lambda x: g.addNode(),
    axis = 1
)

# node_id -> hex_id shorthand dictionary
node_to_hex = {node_id: hex_id for (hex_id, node_id) in hexes_df[['node']].itertuples()}

for (hex_from_id, value_from, node_from, neighbors) in hexes_df[['value', 'node', 'neighbors']].itertuples():
    for hex_to_id in neighbors:
        if hex_to_id in hexes_df.index:
            node_to = hexes_df.at[hex_to_id, 'node']
            value_to = hexes_df.at[hex_to_id, 'value']
            g.addEdge(node_from, node_to, calculate_path_time(value_from, value_to, hex_distance))

g.indexEdges()

In [11]:
# run dijkstra algorithm for all destinations

destinations['hex_id'] = destinations.apply(
    lambda x: h3.geo_to_h3(x['geometry'].x, x['geometry'].y, CONST_graph_detail_lvl),
    axis=1
)
destinations['node_id'] = destinations.apply(
    lambda x: hexes_df.at[x['hex_id'], 'node'],
    axis=1
)


# def calculate_paths(hex_id):
#     dij = nk.distance.Dijkstra(g, hexes_df.at[hex_id, 'node'], True, False)
#     dij.run()
#     return dij

# destinations['dij'] = destinations.swifter.apply(
#     lambda x: calculate_paths(x['hex_id']),
#     axis=1
# )
#destinations.dtypes

In [12]:
# calculate hex id for each destination

paths = []

for (id_from, name_from, hex_from, node_from) in destinations[['name', 'hex_id', 'node_id']].itertuples():

    print('finding paths for destinations {}/{} ({}%)'.format(id_from, len(destinations), int(id_from / len(destinations) * 100)))

    dij = nk.distance.Dijkstra(g, node_from, True, False)
    dij.run()
    for (id_to, name_to, hex_to, node_to) in destinations[['name', 'hex_id', 'node_id']].itertuples():
         

        dist = dij.distance(node_to)
        path = dij.getPath(node_to)
        path_hexes = LineString([hexes_df.at[node_to_hex[i], 'center'] for i in path])

        paths.append(
            {
                "from": name_from,
                "to": name_to,
                "dist": dist,
                "geometry": path_hexes
            }
        )


finding paths for destinations 0/128 (0%)
finding paths for destinations 1/128 (0%)
finding paths for destinations 2/128 (1%)
finding paths for destinations 3/128 (2%)
finding paths for destinations 4/128 (3%)
finding paths for destinations 5/128 (3%)
finding paths for destinations 6/128 (4%)
finding paths for destinations 7/128 (5%)
finding paths for destinations 8/128 (6%)
finding paths for destinations 9/128 (7%)
finding paths for destinations 10/128 (7%)
finding paths for destinations 11/128 (8%)
finding paths for destinations 12/128 (9%)
finding paths for destinations 13/128 (10%)
finding paths for destinations 14/128 (10%)
finding paths for destinations 15/128 (11%)
finding paths for destinations 16/128 (12%)
finding paths for destinations 17/128 (13%)
finding paths for destinations 18/128 (14%)
finding paths for destinations 19/128 (14%)
finding paths for destinations 20/128 (15%)
finding paths for destinations 21/128 (16%)
finding paths for destinations 22/128 (17%)
finding pat

In [13]:
# export paths

paths_df = gpd.GeoDataFrame(paths, crs="epsg:4326")
# simplify geometries
paths_df['geometry'] = paths_df.apply(
      lambda x: wkt.loads(
          wkt.dumps(
              x['geometry'].simplify(0.01, preserve_topology=True), 
              rounding_precision=3
            )
        ),
      axis=1
  )
#paths_df.to_file(PATH_OUT + 'paths.shp', driver="ESRI Shapefile", encoding="utf-8")
paths_df.to_file(PATH_OUT + 'paths.geojson', driver="GeoJSON")

ERROR:fiona._env:/home/adam/projects/itinerary-analysis_premysl-otakar/src/processing/../../data/03_paths/paths.geojson: No such file or directory


In [14]:
# create a destination metrix table

paths_text = "".join([line.strip() for line in open(PATH_OUT + 'paths.geojson')])
paths_dict = json.loads(paths_text)

origins = {}
for fi, feat in enumerate(paths_dict['features']):
    orig = feat['properties']['from'] 
    origins[orig] = {'origin': orig}

for fi, feat in enumerate(paths_dict['features']):
    orig = feat['properties']['from']
    dest = feat['properties']['to']
    dist = feat['properties']['dist']
    origins[orig][dest] = dist

dist_df = pd.DataFrame(origins.values())


dist_df.set_index('origin', inplace=True)
dist_df.to_csv(PATH_OUT + 'dist_m.csv')

print(dist_df.at['Brno', 'Praha'])
print(dist_df.at['Praha', 'Brno'])


8.677852465148167
8.67785246514818
